# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [92]:
def scrape_books(min_rating, max_price):
    url = "http://books.toscrape.com"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    book_names = []
    prices = []
    ratings = []
    upcs = []
    genres = []
    availabilities = []
    descriptions = [] 

    # Find all components
    components = soup.find_all('article', class_= "product_pod")

    for component in components:
        # Extract book name
        book_name = component.find('h3').text.strip()
        book_names.append(book_name)

        # Extract price
        book_price = component.find('p', class_= "price_color").text.strip().replace("£", "")
        prices.append(book_price)

        #Extract rating
        rating_element = component.find('p', class_="star-rating")
        rating = rating_element.get("class")[1]
        if rating == "One":
            final_rating = 1
        elif rating == "Two":
            final_rating = 2
        elif rating == "Three":
            final_rating = 3
        elif rating == "Four":
            final_rating = 4
        elif rating == "Five":
            final_rating = 5
        else:
            final_rating = 0
        ratings.append(final_rating)

        specific_url = component.find('a')
        href = specific_url.get("href")
        full_url = url + "/" + href
    
        #Extract UPC
        response2 = requests.get(full_url)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        upc = soup2.find("td").text.strip()
        upcs.append(upc)

        #Extract genre
        breadcrumb = soup2.find("ul", class_="breadcrumb")
        if breadcrumb:
            category_links = breadcrumb.find_all("a")
            if len(category_links) > 1:
                category = category_links[-1].text.strip()
            else:
                category = "Unknown"
        else:
            category = "Unknown"
        genres.append(category)
        
    
        #Extract availability
        availability_element = component.find('p', class_="instock availability")
        if availability_element:
            first_word = availability_element.text.strip().split()[0] 
            availability = "In stock" if first_word == "In" else "Out of stock"
        else:
            availability = "out of stock"
        availabilities.append(availability)
    
        #Extract description (if avilable)
        description_element = soup2.find("div", id="product_description")
        if description_element:
            description = description_element.find_next("p")
        else:
            description = "No description available"
        descriptions.append(description)

    # Create DataFrame
    df = pd.DataFrame({
        'UPC' : upcs,
        'Book_Name': book_names,
        'Price': prices,
        'Rating' : ratings,
        'Description' : descriptions,
        'Availability' : availabilities,
        'Genre' : genres
    })
    
    df["Price"] = df["Price"].astype(float)
    condition = (df["Rating"]>=min_rating) & (df["Price"]<=max_price)
    return(df[condition])

In [93]:
print(scrape_books(4,20))

                 UPC    Book_Name  Price  Rating  \
12  ce6396b0f23f6ecc  Set Me Free  17.46       5   

                                          Description Availability  \
12  [Aaron Ledbetter’s future had been planned out...     In stock   

          Genre  
12  Young Adult  
